In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **데이터 살펴보기**

In [2]:
import pandas as pd
import warnings 
warnings.filterwarnings(action='ignore')
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import re

In [3]:
#경로 설정
import os
os.chdir('/content/drive/My Drive/Dacon/소설작가분류')

In [4]:
#파일 불러오기
train = pd.read_csv('open/train.csv', encoding = 'utf-8')
test = pd.read_csv('open/test_x.csv', encoding = 'utf-8')
sample_submission = pd.read_csv('open/sample_submission.csv', encoding = 'utf-8')

In [5]:
#train 데이터 살펴보기
train

,index,text,author
0,0,"He was almost choking. There was so much, so m...",3
1,1,"“Your sister asked for it, I suppose?”",2
2,2,"She was engaged one day as she walked, in per...",1
3,3,"The captain was in the porch, keeping himself ...",4
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3
...,...,...,...
54874,54874,"“Is that you, Mr. Smith?” odin whispered. “I h...",2
54875,54875,"I told my plan to the captain, and between us ...",4
54876,54876,"""Your sincere well-wisher, friend, and sister...",1
54877,54877,“Then you wanted me to lend you money?”,3


In [6]:
#test 데이터 살펴보기
test

,index,text
0,0,“Not at all. I think she is one of the most ch...
1,1,"""No,"" replied he, with sudden consciousness, ""..."
2,2,As the lady had stated her intention of scream...
3,3,“And then suddenly in the silence I heard a so...
4,4,His conviction remained unchanged. So far as I...
...,...,...
19612,19612,"At the end of another day or two, odin growing..."
19613,19613,"All afternoon we sat together, mostly in silen..."
19614,19614,"odin, having carried his thanks to odin, proc..."
19615,19615,"Soon after this, upon odin's leaving the room,..."


In [7]:
#sample_submission
sample_submission

,index,0,1,2,3,4
0,0,0,0,0,0,0
1,1,0,0,0,0,0
2,2,0,0,0,0,0
3,3,0,0,0,0,0
4,4,0,0,0,0,0
...,...,...,...,...,...,...
19612,19612,0,0,0,0,0
19613,19613,0,0,0,0,0
19614,19614,0,0,0,0,0
19615,19615,0,0,0,0,0


# **전처리**

In [8]:
#부호를 제거해주는 함수
def alpha_num(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text)

train['text']=train['text'].apply(alpha_num)

In [9]:
#부호가 사라진 것을 확인할 수 있습니다.
train

,index,text,author
0,0,He was almost choking There was so much so muc...,3
1,1,Your sister asked for it I suppose,2
2,2,She was engaged one day as she walked in peru...,1
3,3,The captain was in the porch keeping himself c...,4
4,4,Have mercy gentlemen odin flung up his hands D...,3
...,...,...,...
54874,54874,Is that you Mr Smith odin whispered I hardly d...,2
54875,54875,I told my plan to the captain and between us w...,4
54876,54876,Your sincere wellwisher friend and sister LUC...,1
54877,54877,Then you wanted me to lend you money,3


In [10]:
# 이상치 제거
train = train[train.text != "     "]
train = train.reset_index(drop=True)
train

,index,text,author
0,0,He was almost choking There was so much so muc...,3
1,1,Your sister asked for it I suppose,2
2,2,She was engaged one day as she walked in peru...,1
3,3,The captain was in the porch keeping himself c...,4
4,4,Have mercy gentlemen odin flung up his hands D...,3
...,...,...,...
54831,54874,Is that you Mr Smith odin whispered I hardly d...,2
54832,54875,I told my plan to the captain and between us w...,4
54833,54876,Your sincere wellwisher friend and sister LUC...,1
54834,54877,Then you wanted me to lend you money,3


In [11]:
import nltk
nltk.download('stopwords')
stopwords=nltk.corpus.stopwords.words('english')
stopwords.append('odin')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
# 불용어 제거해주는 함수
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stopwords:
            final_text.append(i.strip())
    return " ".join(final_text)

# 불용어
# stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", 
#              "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", 
#              "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", 
#              "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", 
#              "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", 
#              "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", 
#              "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", 
#              "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", 
#              "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", 
#              "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", 
#              "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

In [13]:
#from nltk.stem.porter import PorterStemmer
#stemmer = PorterStemmer()

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

In [14]:
# 어간추출
def stemming(text):
    final_text = []
    for i in text.split():
      final_text.append(stemmer.stem(i))
    return " ".join(final_text)

In [15]:
#전처리 적용
train['text'] = train['text'].str.lower()
test['text'] = test['text'].str.lower()
train['text'] = train['text'].apply(alpha_num).apply(remove_stopwords).apply(stemming)
test['text'] = test['text'].apply(alpha_num).apply(remove_stopwords).apply(stemming)

In [16]:
# train test 분리
X_train = np.array([x for x in train['text']])
X_test = np.array([x for x in test['text']])
y_train = np.array([x for x in train['author']])

In [17]:
X_train

array(['almost choke much much want say strang exclam came lip pole gaze fix bundl note hand look evid perplex',
       'sister ask suppos',
       'engag one day walk perus jane last letter dwell passag prove jane written spirit instead surpris mr saw look meet put away letter immedi forc smile said',
       ..., 'sincer wellwish friend sister luci', 'want lend money',
       'certain occur said yes like'], dtype='<U1250')

# **모델링**

In [18]:
#파라미터 설정
vocab_size = 20000
embedding_dim = 16
max_length = 500
padding_type='post'
#oov_tok = "<OOV>"

In [19]:
#tokenizer에 fit
tokenizer = Tokenizer(num_words = vocab_size)#, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

In [20]:
#데이터를 sequence로 변환해주고 padding 해줍니다.
train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)

test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)

In [21]:
#가벼운 NLP모델 생성
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [22]:
# compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 16)           320000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 24)                408       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 125       
Total params: 320,533
Trainable params: 320,533
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
# fit model
num_epochs = 20
history = model.fit(train_padded, y_train, 
                    epochs=num_epochs, verbose=2, 
                    validation_split=0.2)

Epoch 1/20
1371/1371 - 7s - loss: 1.5662 - accuracy: 0.2757 - val_loss: 1.5519 - val_accuracy: 0.2683
Epoch 2/20
1371/1371 - 7s - loss: 1.4775 - accuracy: 0.3255 - val_loss: 1.3760 - val_accuracy: 0.3863
Epoch 3/20
1371/1371 - 7s - loss: 1.2507 - accuracy: 0.5090 - val_loss: 1.1601 - val_accuracy: 0.5710
Epoch 4/20
1371/1371 - 7s - loss: 1.0683 - accuracy: 0.5967 - val_loss: 1.0339 - val_accuracy: 0.6061
Epoch 5/20
1371/1371 - 7s - loss: 0.9514 - accuracy: 0.6409 - val_loss: 0.9506 - val_accuracy: 0.6399
Epoch 6/20
1371/1371 - 7s - loss: 0.8707 - accuracy: 0.6722 - val_loss: 0.8996 - val_accuracy: 0.6583
Epoch 7/20
1371/1371 - 7s - loss: 0.8130 - accuracy: 0.6936 - val_loss: 0.8702 - val_accuracy: 0.6607
Epoch 8/20
1371/1371 - 7s - loss: 0.7655 - accuracy: 0.7105 - val_loss: 0.8684 - val_accuracy: 0.6572
Epoch 9/20
1371/1371 - 7s - loss: 0.7287 - accuracy: 0.7240 - val_loss: 0.8213 - val_accuracy: 0.6844
Epoch 10/20
1371/1371 - 7s - loss: 0.6959 - accuracy: 0.7376 - val_loss: 0.8069 - 

In [24]:
# predict values
pred = model.predict_proba(test_padded)

Instructions for updating:
Please use `model.predict()` instead.


In [25]:
pred

array([[2.5447593e-05, 8.1620926e-01, 1.8368286e-01, 4.8815993e-05,
        3.3573851e-05],
       [6.5618522e-02, 5.2875213e-02, 6.0934049e-01, 2.5617262e-02,
        2.4654853e-01],
       [9.9502748e-01, 2.1498249e-04, 2.2333757e-06, 1.8686758e-07,
        4.7550732e-03],
       ...,
       [1.0620578e-04, 9.9983132e-01, 6.1844090e-05, 5.5786927e-07,
        5.0824834e-08],
       [1.6547396e-04, 9.9979383e-01, 4.0123949e-05, 5.5201582e-07,
        4.7674384e-08],
       [9.9890089e-01, 2.1083334e-08, 2.9728453e-09, 5.6927756e-04,
        5.2976876e-04]], dtype=float32)

In [26]:
# submission
sample_submission[['0','1','2','3','4']] = pred
sample_submission

,index,0,1,2,3,4
0,0,2.544759e-05,8.162093e-01,1.836829e-01,4.881599e-05,3.357385e-05
1,1,6.561852e-02,5.287521e-02,6.093405e-01,2.561726e-02,2.465485e-01
2,2,9.950275e-01,2.149825e-04,2.233376e-06,1.868676e-07,4.755073e-03
3,3,9.659421e-08,3.705055e-06,9.956769e-01,2.324286e-05,4.295966e-03
4,4,9.882970e-01,4.652491e-05,1.853154e-06,1.094450e-02,7.101051e-04
...,...,...,...,...,...,...
19612,19612,4.556071e-06,9.999955e-01,1.690839e-17,2.076829e-15,5.804496e-21
19613,19613,2.591840e-02,1.692985e-08,3.651359e-06,5.160883e-08,9.740779e-01
19614,19614,1.062058e-04,9.998313e-01,6.184409e-05,5.578693e-07,5.082483e-08
19615,19615,1.654740e-04,9.997938e-01,4.012395e-05,5.520158e-07,4.767438e-08


In [28]:
sample_submission.to_csv('submission/snow_submission02.csv', index = False, encoding = 'utf-8')